In [1]:
#double と int のほうが馴染む……馴染まない?
double = Float64
int = Int64

Int64

## struct の設定


In [2]:
# 次元指定なし
struct Fields
  ϕ ::Array{double}
  u ::Array{double}
  v ::Array{double}
end

# 次元指定あり
struct Fields2D
  ϕ ::Array{double,2}
  u ::Array{double,2}
  v ::Array{double,2}
end

# 以下コンストラクタ
function Fields(Nx,Ny)
  ϕ = Array{double,2}(undef,Nx+2,Ny+2)
  u = copy(ϕ)
  v = copy(ϕ)
  return Fields(ϕ,u,v)
end
function Fields2D(Nx,Ny)
  ϕ = Array{double,2}(undef,Nx+2,Ny+2)
  u = copy(ϕ)
  v = copy(ϕ)
  return Fields2D(ϕ,u,v)
end

Fields2D

In [3]:
"計算の初期条件設定"
function syoki(field)
  field.ϕ .= 0.
  field.u .=1
  field.v .=1
  
  field.ϕ[2:51, 2:101] .= 1.
end

syoki

In [4]:

Nx = 300; Ny = 150
F = Fields(Nx,Ny)
F2 = Fields2D(Nx,Ny);
syoki(F)
syoki(F2);



In [5]:
"2次元のの移流方程式"
function t1_x1_kazakami(field;dx=1,dt=1e-1,T=500,step=2)
  Nx,Ny = size(field.ϕ)
  
  Bx = 2:Nx-1; By = 2:Ny-1
  dϕ = copy(field.ϕ)
  Lϕ = Array{Float64,3}(undef,Nx,Ny,div(length(0:dt:T),step)+1)
  
  
  for(i,t) in enumerate(0:dt:T)
    
    #Periodic Boundary Condition
    #boundary
    #edge
    for x in Bx
      field.ϕ[x,1] = field.ϕ[x,Ny-1]
      field.ϕ[x,Ny] = field.ϕ[x,2]
    end
    for y in By
      field.ϕ[1,y] = field.ϕ[Nx-1,y]
      field.ϕ[Nx,y] = field.ϕ[2,y]
    end
    
    #corner
    field.ϕ[1,1]=field.ϕ[Nx-1,Ny-1]
    field.ϕ[Nx,1]=field.ϕ[2,Ny-1]
    field.ϕ[1,Ny]=field.ϕ[Nx-1,2]
    field.ϕ[Nx,Ny]=field.ϕ[2,2]
    
    for y in By, x in Bx
      dϕ[x,y] = (
        field.u[x,y] * 1/dx * (field.ϕ[x,y]-field.ϕ[x-1,y]) 
        + field.v[x,y]  * 1/dx * (field.ϕ[x,y]-field.ϕ[x,y-1])
        )
    end
    
    for y in By, x in Bx
      field.ϕ[x,y] -= dt*dϕ[x,y]
    end
    
    if i%step ==1
      Lϕ[:,:,div(i,step)+1] .= field.ϕ
    end
    if i%(div(length(0:dt:T),10)) ==1
      #println(i, "//",length(0:dt:T) )
    end
  end
end

t1_x1_kazakami

# 比較
同じ関数にそれぞれの構造体を投げたときの時間比較

In [6]:
println("次元設定なし")
@time L = t1_x1_kazakami(F;dt=1e-2,T=1)
@time L = t1_x1_kazakami(F;dt=1e-2,T=1)

println("次元指定あり")
@time L2 = t1_x1_kazakami(F2;dt=1e-2,T=1)
@time L2 = t1_x1_kazakami(F2;dt=1e-2,T=1)

次元設定なし
  2.836653 seconds (101.62 M allocations: 1.720 GiB, 6.13% gc time)
  2.369190 seconds (100.16 M allocations: 1.647 GiB, 2.16% gc time)
次元指定あり
  0.126995 seconds (137.76 k allocations: 24.872 MiB)
  0.036511 seconds (65 allocations: 18.215 MiB, 10.54% gc time)
